This notebook generates customer-level features that will help us train a machine learning model to predict churn.

In [51]:
# Import required libraries
import pandas as pd
import numpy as np

In [52]:
#Load the Merged Dataset

order_full = pd.read_pickle("outputs/order_full.pkl")
order_full.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,product_category_name_english
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,housewares
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,housewares
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,housewares
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,...,178.0,1.0,400.0,19.0,13.0,19.0,31570.0,belo horizonte,SP,perfumery
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,...,232.0,1.0,420.0,24.0,19.0,21.0,14840.0,guariba,SP,auto


In [53]:
# Filter Only Delivered Orders (for churn calculation)
order_full = order_full[order_full['order_status'] == 'delivered']
order_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 115723 entries, 0 to 119142
Data columns (total 40 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       115723 non-null  object 
 1   customer_id                    115723 non-null  object 
 2   order_status                   115723 non-null  object 
 3   order_purchase_timestamp       115723 non-null  object 
 4   order_approved_at              115708 non-null  object 
 5   order_delivered_carrier_date   115721 non-null  object 
 6   order_delivered_customer_date  115715 non-null  object 
 7   order_estimated_delivery_date  115723 non-null  object 
 8   customer_unique_id             115723 non-null  object 
 9   customer_zip_code_prefix       115723 non-null  int64  
 10  customer_city                  115723 non-null  object 
 11  customer_state                 115723 non-null  object 
 12  order_item_id                  1157

Now Generating Customer-Level Features

In [54]:
#Aggregate key stats per customer
customer_features = order_full.groupby('customer_unique_id').agg({
    'order_id': 'count',                                # total orders
    'payment_value': 'sum',                             # total amount spent
    'review_score': 'mean',                             # average review score
    'order_purchase_timestamp': ['min', 'max']          # first and last purchase dates
}).reset_index()

customer_features.head()

customer_unique_id order_id payment_value review_score  \
                                       count           sum         mean   
0  0000366f3b9a7992bf8c76cfdf3221e2        1        141.90          5.0   
1  0000b849f77a49e4a4ce2b2a4ca5be3f        1         27.19          4.0   
2  0000f46a3911fa3c0805444483337064        1         86.22          3.0   
3  0000f6ccb0745a6a4b88665a16c9f078        1         43.62          4.0   
4  0004aac84e0df4da2b147fca70cf8255        1        196.89          5.0   

  order_purchase_timestamp                       
                       min                  max  
0      2018-05-10 10:56:27  2018-05-10 10:56:27  
1      2018-05-07 11:11:27  2018-05-07 11:11:27  
2      2017-03-10 21:05:03  2017-03-10 21:05:03  
3      2017-10-12 20:29:41  2017-10-12 20:29:41  
4      2017-11-14 19:45:42  2017-11-14 19:45:42

In [55]:
# Flatten MultiIndex Columns
customer_features.columns = ['customer_unique_id', 'total_orders', 'total_spend', 
                             'avg_review_score', 'first_purchase', 'last_purchase']
customer_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93358 entries, 0 to 93357
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customer_unique_id  93358 non-null  object 
 1   total_orders        93358 non-null  int64  
 2   total_spend         93358 non-null  float64
 3   avg_review_score    92755 non-null  float64
 4   first_purchase      93358 non-null  object 
 5   last_purchase       93358 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 4.3+ MB


Adding Date-Based Features

In [56]:
# Days Since Last Purchase (assuming data ends at 2018-10-17)

customer_features['last_purchase'] = pd.to_datetime(customer_features['last_purchase'])
latest_date = pd.to_datetime('2018-10-17')
customer_features['days_since_last_purchase'] = (latest_date - customer_features['last_purchase']).dt.days

In [57]:
# Average Days Between Orders

customer_features['first_purchase'] = pd.to_datetime(customer_features['first_purchase'])
customer_features['avg_days_between_orders'] = (
    (customer_features['last_purchase'] - customer_features['first_purchase']).dt.days /
    (customer_features['total_orders'] - 1).replace(0, 1)
)

In [58]:
# Define Churn Label
# Customers who haven't purchased in the last 180 days are labeled as churned.

customer_features['is_churned'] = (customer_features['days_since_last_purchase'] > 180).astype(int)

In [59]:
# Final Cleanup

# Drop columns not needed for modeling
final_data = customer_features.drop(columns=[
    'first_purchase', 'last_purchase', 'days_since_last_purchase'
])
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93358 entries, 0 to 93357
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   customer_unique_id       93358 non-null  object 
 1   total_orders             93358 non-null  int64  
 2   total_spend              93358 non-null  float64
 3   avg_review_score         92755 non-null  float64
 4   avg_days_between_orders  93358 non-null  float64
 5   is_churned               93358 non-null  int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 4.3+ MB


In [60]:
# Separate features and target
X = final_data.drop('is_churned', axis=1)
y = final_data['is_churned']

In [61]:
print("✅ Missing values after filling:\n", X.isnull().sum())

✅ Missing values after filling:
 customer_unique_id           0
total_orders                 0
total_spend                  0
avg_review_score           603
avg_days_between_orders      0
dtype: int64


In [62]:
# Fill missing values with a neutral score, e.g., 3.0
X['avg_review_score'].fillna(3.0, inplace=True)


C:\Users\asif\AppData\Local\Temp\ipykernel_7228\2836961652.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['avg_review_score'].fillna(3.0, inplace=True)


In [63]:
print("✅ Missing values after filling:\n", X.isnull().sum())


✅ Missing values after filling:
 customer_unique_id         0
total_orders               0
total_spend                0
avg_review_score           0
avg_days_between_orders    0
dtype: int64


In [64]:
X = pd.read_pickle("outputs/X.pkl")
y = pd.read_pickle("outputs/y.pkl")

# Fill missing values before modeling
X['avg_review_score'].fillna(X['avg_review_score'].mean(), inplace=True)


C:\Users\asif\AppData\Local\Temp\ipykernel_7228\1004095133.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['avg_review_score'].fillna(X['avg_review_score'].mean(), inplace=True)


In [65]:
# Save for Modeling

X.to_pickle("outputs/X.pkl")
y.to_pickle("outputs/y.pkl")